In [814]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import pandas as pd
import numpy as np
import re
from string import digits
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_regression
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error
import sklearn
import shap
import pandas as pd

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Load Data

In [815]:
path_file = "C:/Users/StefanPopov/Downloads/Python hackathon_Mock data_Group 3 .xlsx"

df = pd.read_excel(path_file, sheet_name='Наеми', header=2)
df=df.drop_duplicates()
df



,Обява №,Дата на публикуване,Заглавие,Описание,Област,Град,Район/Квартал,Етаж,Наем (в лв.),Наем (в евро),Площ,Цена на кв.м. (в лв.),Спални,От агенция,Подател,Телефон,Идентифициран риск за фалшива обява
0,1276,2022-09-25,Дава под Наем 1-СТАЕН,ЕКСКЛУЗИВНА ОФЕРТА\n\n'БЕСТ СЕЛЪРС' предлага н...,София,София,Изток,8,NaN,275.0,49,10.976597,1,Да,Агенция: БЕСТ СЕЛЪРС ООД,886224488,Не
1,1388,2022-09-26,Дава под Наем 1-СТАЕН,Топ локация!!!\n\nСлед ремонт - напълно обзаве...,София,София,Белите брези,NaN,450,NaN,40,11.250000,1,Да,Агенция: УПРАВЛЕНИЕ БГ,884231053,Не
2,1500,2022-09-27,Дава под Наем 1-СТАЕН,"Предлагаме малко,но функционално жилище,в мног...",София,София,Редута,0,400,NaN,30,13.330000,1,Да,Агенция: АГЕНЦИИ ИЗТОК - ОФИС ТОВИСТО М,988829752,Не
3,1612,2022-09-28,Дава под Наем 1-СТАЕН,На вашето внимание агенция за недвижими имоти ...,София,София,Разсадника,2,NaN,250.0,47,10.463690,1,Да,Агенция: ПИЙК РИЪЛ ЕСТЕЙТ ЕООД,882988975,Не
4,1724,2022-09-29,Дава под Наем 1-СТАЕН,БЕЗ ХАЗАЙ на спирката с/у Медицинска а-мия. Ст...,София,София,Медицинска ададемия,1,380,NaN,25,15.200000,1,Да,Агенция:ЛАЙТ ПРОПЪРТИ,878318739,Да
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,5049,2022-09-25,Дава под Наем 2-СТАЕН,"ОБЗАВЕДЕН, ТОПЪЛ на комуникативно място във въ...",София,София,Западен парк,4,500,NaN,48,10.420000,1,Да,Агенция: УПРАВЛЕНИЕ БГ,884231053,Не
94,5094,2022-09-26,Дава под Наем 2-СТАЕН,На вашето внимание агенция за недвижими имоти ...,София,София,Люлин 5,13,NaN,250.0,59,8.292719,1,Да,Агенция: ПИЙК РИЪЛ ЕСТЕЙТ ЕООД,882988975,Не
95,5139,2022-09-27,Дава под Наем 1-СТАЕН,"Отличен, функционален дом! Състои се от: самос...",София,София,Лозенец,2,NaN,200.0,50,7.823320,1,Да,Агенция: Василев,887069908,Не
96,5184,2022-09-28,Дава под Наем 1-СТАЕН,Предлагаме Ви самостоятелна къща в кв. Левски....,София,София,Левски Г,4,450,NaN,65,6.920000,1,Да,Агенция: Адрес,889970603,Не


Изчистваме дейтасета от колони, които не биха донесли никаква стойност

In [816]:
dropColumns = ['Обява №', 'Етаж', 'Телефон', 'Спални', 'Дата на публикуване', 'Област', 'Град', 'Район/Квартал','От агенция', 'Подател', 'Заглавие']
df=df.drop(dropColumns,axis=1)



Тъй като на места имаме цена в евро, а на други в лева - уеднаквяваме всички суми в лева, след което махаме колоната с евро

In [817]:
df["Наем (в лв.)"] = df["Наем (в лв.)"].replace('', pd.NA).fillna((df['Наем (в евро)'])*2)
#df["Наем (в лв.)"] = df["Наем (в лв.)"].str.strip()
#df['Наем (в лв.)'] = df['Наем (в лв.)'].astype(int)
for i in range(len(df["Наем (в лв.)"])):
    pos = []
    if(type(df["Наем (в лв.)"][i]) is str):
        print("here")
        
        df = df.drop(i)
#df.drop(df[pos], inplace=True)
    

#df["Наем (в лв.)"] = pd.to_numeric(df["Наем (в лв.)"], downcast="float")
df = df.drop('Наем (в евро)', axis=1)
df

here
here


,Описание,Наем (в лв.),Площ,Цена на кв.м. (в лв.),Идентифициран риск за фалшива обява
0,ЕКСКЛУЗИВНА ОФЕРТА\n\n'БЕСТ СЕЛЪРС' предлага н...,550,49,10.976597,Не
1,Топ локация!!!\n\nСлед ремонт - напълно обзаве...,450,40,11.250000,Не
2,"Предлагаме малко,но функционално жилище,в мног...",400,30,13.330000,Не
3,На вашето внимание агенция за недвижими имоти ...,500,47,10.463690,Не
4,БЕЗ ХАЗАЙ на спирката с/у Медицинска а-мия. Ст...,380,25,15.200000,Да
...,...,...,...,...,...
93,"ОБЗАВЕДЕН, ТОПЪЛ на комуникативно място във въ...",500,48,10.420000,Не
94,На вашето внимание агенция за недвижими имоти ...,500,59,8.292719,Не
95,"Отличен, функционален дом! Състои се от: самос...",400,50,7.823320,Не
96,Предлагаме Ви самостоятелна къща в кв. Левски....,450,65,6.920000,Не


Изчистваме всички символи от колоната с описания. Използваме BOW, за да преброим точно колко пъти са се повтаряли едни и същи думи в различните обяви. Освен това създаваме нова колона, която да съдържа точната бройка на думите, използвани в обявата (Може би измамниците имат тенденция да използват повече/по-малко думи в обявата)

In [818]:


from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
df['Описание'] = df['Описание'].str.lower()
df['Описание'] = df['Описание'].str.replace(' ', 'SPACE')
df['Описание'] = df['Описание'].str.replace('\n', 'NEWLINE')
df['Описание'] = df['Описание'].str.replace('\W', '')
df['Описание'] = df['Описание'].str.replace('SPACE', ' ')
df['Описание'] = df['Описание'].str.replace('NEWLINE', ' ')
remove_digits = str.maketrans('', '', digits)
df['Описание'] = df['Описание'].str.translate(remove_digits)
words = df['Описание'].str.translate(remove_digits)
#print(words)
df['Брой Думи'] = (df['Описание'].str.split().str.len())
#print(df)
X = vectorizer.fit_transform(df['Описание'])
#print(X)
df['Описание'] = X.toarray().tolist()
df2 = pd.DataFrame(pd.DataFrame(df['Описание'].tolist()))
df = pd.concat([df, df2], axis=1)
df = df.dropna()

Използвам Label Encoder, за да обърна Не/Да в 0,1. Дропвам колоните Описание, защото за всяка една дума в тази колона правя нова колона. Тази нова колона съдържа броят пъти, в който дума се среща в описанието. Дропваме и 'Идентифициран риск за фалшива обява', защото вече сме ги запазили в променливата fake.

In [819]:
from sklearn import preprocessing
  

label_encoder = preprocessing.LabelEncoder()
  

fake= label_encoder.fit_transform(df['Идентифициран риск за фалшива обява'].str.strip())

df = df.drop('Описание', axis=1)
df = df.drop('Идентифициран риск за фалшива обява', axis=1)
#df = df.drop('Брой Думи', axis=1)
#df = df.drop('Наем (в лв.)', axis=1)

#df = df.to_numpy()
#df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df

,Наем (в лв.),Площ,Цена на кв.м. (в лв.),Брой Думи,0,1,2,3,4,5,...,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362
0,550,49.0,10.976597,90.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,450,40.0,11.250000,35.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,400,30.0,13.330000,54.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,500,47.0,10.463690,86.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,380,25.0,15.200000,42.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,530,70.0,7.570000,34.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
92,500,56.0,8.930000,17.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,500,48.0,10.420000,66.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,500,59.0,8.292719,82.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Изпозлваме RandomForestClassifier 

In [820]:
X_train, X_test, y_train, y_test = train_test_split(df, fake, test_size=0.2)

from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)
print(X_train)
print(y_train)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

predictions=clf.predict(X_test)
results=0
for i in range(len(y_test)):
    print("Actual: " + str(y_test[i]))
    print("Prediction: " + str(predictions[i]))

for i in range(len(predictions)):
    results = results + (predictions[i] - y_test[i])**2

    #meanPercentageError

print ("Mean Abs: " + str(mean_absolute_error(y_test,predictions)))

   Наем (в лв.)  Площ  Цена на кв.м. (в лв.)  Брой Думи    0    1    2    3  \
65          350  46.0               7.610000       73.0  0.0  0.0  0.0  0.0   
15          399  40.0               9.970000      102.0  0.0  0.0  0.0  0.0   
21          320  29.0              10.796182       60.0  0.0  0.0  0.0  0.0   
41          500  35.0              13.964626       67.0  0.0  0.0  0.0  0.0   
68          400  33.0              12.120000       53.0  0.0  1.0  0.0  0.0   
..          ...   ...                    ...        ...  ...  ...  ...  ...   
22          480  50.0               9.600000       62.0  0.0  0.0  0.0  0.0   
37          550  48.0              11.206906      103.0  0.0  0.0  0.0  0.0   
30          500  40.0              12.223937       42.0  0.0  0.0  0.0  0.0   
32          480  50.0               9.387984       79.0  0.0  0.0  0.0  0.0   
78          548  45.0              11.911005       35.0  0.0  0.0  1.0  0.0   

      4    5  ...  1353  1354  1355  1356  1357  13